In [32]:
import utils as utils
from nltk.classify import NaiveBayesClassifier

In [33]:
DATA_FILE_PATH = 'data.json'

In [34]:
train_tups, test_tups = utils.split_data(utils.generate_tuples_from_file(DATA_FILE_PATH), test_size=0.2)

print('Training set size: ', len(train_tups[0]))
print('Test set size: ', len(test_tups[0]))
print('Sample tokenized review: ', train_tups[0][0])
print('Sample rating: ', train_tups[1][0])

Training set size:  7
Test set size:  2
Sample tokenized review:  ['Cute', 'interior', 'and', 'owner', '(', '?', ')', 'gave', 'us', 'tour', 'of', 'upcoming', 'patio/rooftop', 'area', 'which', 'will', 'be', 'great', 'on', 'beautiful', 'days', 'like', 'today', '.', 'Cheese', 'curds', 'were', 'very', 'good', 'and', 'very', 'filling', '.', 'Really', 'like', 'that', 'sandwiches', 'come', 'w', 'salad', ',', 'esp', 'after', 'eating', 'too', 'many', 'curds', '!', 'Had', 'the', 'onion', ',', 'gruyere', ',', 'tomato', 'sandwich', '.', 'Was', "n't", 'too', 'much', 'cheese', 'which', 'I', 'liked', '.', 'Needed', 'something', 'else', '...', 'pepper', 'jelly', 'maybe', '.', 'Would', 'like', 'to', 'see', 'more', 'menu', 'options', 'added', 'such', 'as', 'salads', 'w', 'fun', 'cheeses', '.', 'Lots', 'of', 'beer', 'and', 'wine', 'as', 'well', 'as', 'limited', 'cocktails', '.', 'Next', 'time', 'I', 'will', 'try', 'one', 'of', 'the', 'draft', 'wines', '.']
Sample rating:  4


In [35]:
def word_feats(list_of_words) -> dict:    
    return dict([(word, True) for word in list_of_words])

def word_feats_mult(list_of_words) -> dict:
    return dict([(word, list_of_words.count(word)) for word in list_of_words])

In [36]:
labeled_feature_sets = []
for i in range(len(train_tups[0])):
    feature_set = word_feats(train_tups[0][i])
    _class = train_tups[1][i]
    labeled_feature_sets.append((feature_set, _class))

classifier = NaiveBayesClassifier.train(labeled_featuresets=labeled_feature_sets)

In [37]:
unseen_review = word_feats_mult('The food was absolutely terrible.')
rating_prediction = classifier.classify(classifier)

print('Rating prediction: ', rating_prediction)

AttributeError: 'NaiveBayesClassifier' object has no attribute 'copy'